In [36]:
import pandas as pd

In [37]:
# Ruta del dataset
ruta_dataset = "Limpieza/data/df_unificado_limpio_con_proyecciones_limpio.csv"

In [38]:
df = pd.read_csv(ruta_dataset)

In [39]:
print("Estructura del dataset:")
print(df.info())

Estructura del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 917690 entries, 0 to 917689
Data columns (total 14 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   AÑO                           917690 non-null  int64  
 1   TRIMESTRE                     917690 non-null  float64
 2   PROVEEDOR                     917690 non-null  object 
 3   CÓDIGO DANE                   917690 non-null  float64
 4   DEPARTAMENTO                  917690 non-null  object 
 5   CÓDIGO DANE.1                 917690 non-null  float64
 6   MUNICIPIO                     917690 non-null  object 
 7   SEGMENTO                      917690 non-null  object 
 8   TECNOLOGÍA                    917690 non-null  object 
 9   VELOCIDAD BAJADA              917690 non-null  float64
 10  VELOCIDAD SUBIDA              917690 non-null  float64
 11  No. ACCESOS FIJOS A INTERNET  917690 non-null  int64  
 12  Latitud             

In [40]:
print("\nValores nulos por columna:")
print(df.isnull().sum())


Valores nulos por columna:
AÑO                             0
TRIMESTRE                       0
PROVEEDOR                       0
CÓDIGO DANE                     0
DEPARTAMENTO                    0
CÓDIGO DANE.1                   0
MUNICIPIO                       0
SEGMENTO                        0
TECNOLOGÍA                      0
VELOCIDAD BAJADA                0
VELOCIDAD SUBIDA                0
No. ACCESOS FIJOS A INTERNET    0
Latitud                         0
Longitud                        0
dtype: int64


In [41]:
total_filas_nulas = df.isnull().any(axis=1).sum()
print(f"\nTotal de filas con valores nulos: {total_filas_nulas}")


Total de filas con valores nulos: 0


In [42]:
for columna in ['PROVEEDOR', 'SEGMENTO', 'TECNOLOGÍA']:
    moda = df[columna].mode()[0]  # Obtener el valor más frecuente
    df[columna].fillna(moda, inplace=True)

/tmp/ipykernel_93638/2239629596.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[columna].fillna(moda, inplace=True)


In [43]:
for columna in ['TRIMESTRE', 'VELOCIDAD BAJADA', 'VELOCIDAD SUBIDA', 'Latitud', 'Longitud']:
    media = df[columna].mean()  # Calcular la media
    df[columna].fillna(media, inplace=True)

/tmp/ipykernel_93638/1880521349.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[columna].fillna(media, inplace=True)


In [44]:
for index, row in df[df['CÓDIGO DANE'].isnull()].iterrows():
    departamento = row['DEPARTAMENTO']
    municipio = row['MUNICIPIO']

    # Buscar valores únicos del dataset para este municipio y departamento
    codigo_dane = df[(df['DEPARTAMENTO'] == departamento) & (df['MUNICIPIO'] == municipio)]['CÓDIGO DANE'].dropna().unique()
    codigo_dane_1 = df[(df['DEPARTAMENTO'] == departamento) & (df['MUNICIPIO'] == municipio)]['CÓDIGO DANE.1'].dropna().unique()

    # Imputar si se encuentra un único valor
    if len(codigo_dane) == 1:
        df.loc[index, 'CÓDIGO DANE'] = codigo_dane[0]
    if len(codigo_dane_1) == 1:
        df.loc[index, 'CÓDIGO DANE.1'] = codigo_dane_1[0]


In [45]:
print("\nValores nulos restantes por columna:")
print(df.isnull().sum())


Valores nulos restantes por columna:
AÑO                             0
TRIMESTRE                       0
PROVEEDOR                       0
CÓDIGO DANE                     0
DEPARTAMENTO                    0
CÓDIGO DANE.1                   0
MUNICIPIO                       0
SEGMENTO                        0
TECNOLOGÍA                      0
VELOCIDAD BAJADA                0
VELOCIDAD SUBIDA                0
No. ACCESOS FIJOS A INTERNET    0
Latitud                         0
Longitud                        0
dtype: int64


In [25]:
ruta_dataset_imputado = "Limpieza/data/df_unificado_limpio_imputado.csv"
df.to_csv(ruta_dataset_imputado, index=False)
print(f"Dataset imputado guardado en: {ruta_dataset_imputado}")

Dataset imputado guardado en: Limpieza/data/df_unificado_limpio_imputado.csv


In [26]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [27]:
columnas_predictoras = ['AÑO', 'TRIMESTRE', 'VELOCIDAD BAJADA', 'VELOCIDAD SUBIDA', 'Latitud', 'Longitud']
X = df[columnas_predictoras]
y = df['No. ACCESOS FIJOS A INTERNET']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [30]:
y_pred = modelo.predict(X_test)

In [31]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [32]:
print(f"Error cuadrático medio (MSE): {mse}")
print(f"Coeficiente de determinación (R²): {r2}")

Error cuadrático medio (MSE): 388123.41052680725
Coeficiente de determinación (R²): 0.00042247144017781846


In [33]:
print("Coeficientes del modelo:")
for feature, coef in zip(columnas_predictoras, modelo.coef_):
    print(f"{feature}: {coef}")


Coeficientes del modelo:
AÑO: 8.61687203473217
TRIMESTRE: 4.914257424981054
VELOCIDAD BAJADA: 0.0005861939713001085
VELOCIDAD SUBIDA: -0.0006051435344360812
Latitud: 3.999326016621527
Longitud: -2.274157728662584


Conclusiones del Modelo de Regresión Lineal
1. Desempeño del modelo
Error Cuadrático Medio (MSE): 388123.41
El modelo presenta un error promedio elevado, indicando que las predicciones no están cerca de los valores reales.
Coeficiente de Determinación (R²): 0.00042
Solo el 0.04% de la variabilidad en los accesos fijos a Internet puede ser explicada por las variables predictoras seleccionadas.
Esto indica que el modelo tiene un poder explicativo extremadamente bajo.
2. Análisis de los coeficientes
AÑO: El modelo predice que por cada año adicional, los accesos fijos a Internet aumentan en 8.61 unidades. Este incremento es insignificante frente al rango de datos.
TRIMESTRE: Tiene un efecto marginal, con un cambio de 4.91 unidades en promedio.
VELOCIDAD BAJADA y SUBIDA: Tienen coeficientes pequeños (+0.00058 y -0.00060, respectivamente), lo que indica que su influencia sobre los accesos fijos es mínima.
Latitud y Longitud: Tienen impactos mínimos (+3.99 y -2.27 respectivamente), lo que sugiere que la ubicación geográfica, representada de esta forma, no es un factor significativo.
3. Limitaciones del modelo
Bajo R²: El modelo lineal no capta las relaciones subyacentes entre las variables predictoras y el objetivo.
Variables predictoras insuficientes o no relevantes:
Las variables incluidas no explican suficientemente la variación en los accesos fijos.
Podría ser necesario incluir variables más representativas, como infraestructura tecnológica o características demográficas.
Suposición de linealidad: La relación entre las variables predictoras y el objetivo podría no ser lineal, lo que limita la efectividad de este modelo.


In [34]:
df

,AÑO,TRIMESTRE,PROVEEDOR,CÓDIGO DANE,DEPARTAMENTO,CÓDIGO DANE.1,MUNICIPIO,SEGMENTO,TECNOLOGÍA,VELOCIDAD BAJADA,VELOCIDAD SUBIDA,No. ACCESOS FIJOS A INTERNET,Latitud,Longitud
0,2021,3.000000,@DIGITAL GROUP SAS,5.0,ANTIOQUIA,5031.0,AMALFI,CORPORATIVO,FIBER TO THE HOME (FTTH),20.00000,5.00000,1,6.895627,-75.014382
1,2021,3.000000,@DIGITAL GROUP SAS,5.0,ANTIOQUIA,5031.0,AMALFI,CORPORATIVO,FIBER TO THE HOME (FTTH),30.00000,5.00000,1,6.895627,-75.014382
2,2021,3.000000,@DIGITAL GROUP SAS,5.0,ANTIOQUIA,5031.0,AMALFI,RESIDENCIAL - ESTRATO 1,FIBER TO THE HOME (FTTH),18.00000,5.00000,1,6.895627,-75.014382
3,2021,3.000000,@DIGITAL GROUP SAS,5.0,ANTIOQUIA,5088.0,BELLO,RESIDENCIAL - ESTRATO 2,FIBER TO THE HOME (FTTH),30.00000,5.00000,3,6.336729,-75.559589
4,2021,3.000000,@DIGITAL GROUP SAS,5.0,ANTIOQUIA,5088.0,BELLO,RESIDENCIAL - ESTRATO 2,FIBER TO THE HOME (FTTH),8.00000,4.00000,10,6.336729,-75.559589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
917880,2025,2.457586,COMUNICACION CELULAR S A COMCEL S A,76.0,VALLE DEL CAUCA,76869.0,VIJES,CORPORATIVO,CABLE,204.69476,141.51949,1249,6.196477,-74.789362
917881,2025,2.457586,COMUNICACION CELULAR S A COMCEL S A,76.0,VALLE DEL CAUCA,76890.0,YOTOCO,CORPORATIVO,CABLE,204.69476,141.51949,1618,6.196477,-74.789362
917882,2025,2.457586,COMUNICACION CELULAR S A COMCEL S A,76.0,VALLE DEL CAUCA,76892.0,YUMBO,CORPORATIVO,CABLE,204.69476,141.51949,21173,6.196477,-74.789362
917883,2025,2.457586,COMUNICACION CELULAR S A COMCEL S A,76.0,VALLE DEL CAUCA,76895.0,ZARZAL,CORPORATIVO,CABLE,204.69476,141.51949,7209,6.196477,-74.789362
